In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [3]:
import os
import sys
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import googletrans
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [66]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [67]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


In [68]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: job_description.replace('incl.', 'including')
)


In [69]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: job_description.replace('e.g.', 'e.g')
)


In [70]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: job_description.replace('e.g', 'for example')
)


In [71]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: job_description.replace('etc.', 'et cetera')
)


In [6]:
# df_jobs = df_jobs[:100]

In [7]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [8]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [73]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64684 entries, 0 to 64683
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     64684 non-null  object 
 1   Platform           64684 non-null  object 
 2   Job ID             64684 non-null  object 
 3   Job Title          64684 non-null  object 
 4   Company Name       64679 non-null  object 
 5   Location           64684 non-null  object 
 6   Job Description    64684 non-null  object 
 7   Rating             4130 non-null   float64
 8   Employment Type    63959 non-null  object 
 9   Company URL        61360 non-null  object 
 10  Job URL            64684 non-null  object 
 11  Job Age            64684 non-null  object 
 12  Job Age Number     64684 non-null  object 
 13  Collection Date    64684 non-null  object 
 14  Data Row           60551 non-null  float64
 15  Tracking ID        60551 non-null  object 
 16  Industry           612

In [74]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [87]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope'



In [88]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


2

In [89]:
# job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [96]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [97]:
len(job_descriptions)


2

In [98]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
job_descriptions[0][1500:2000]

'nage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.\n\nCommunication   stakeholder management\nAct as the key contact person for strategic stakeholders.\nApply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.\nAs conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.\nDrive chang'

In [100]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'



In [101]:
# nltk_sentencizer = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencizer.append(sent)

In [102]:
nltk_sentencizer = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [103]:
len(nltk_sentencizer)

133

In [104]:
for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_eg in sent:
        print(idx)

17


In [105]:
next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_eg in sent)

17

In [106]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

20
84


In [107]:
# sentences split on e.g. and incl.
nltk_sentencizer[
    next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_eg in sent):next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_incl.split('\(')[0] in sent)+2
]


['Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.']

In [108]:
for idx, sent in enumerate(nltk_sentencizer):
    if 'Power' in sent:
        print(idx)

62
128


In [109]:
next(idx for idx, sent in enumerate(nltk_sentencizer) if 'Power' in sent)

62

In [110]:
nltk_sentencizer[next(idx for idx, sent in enumerate(nltk_sentencizer) if 'Power' in sent)]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [112]:
for idx, sentence in enumerate(nltk_sentencizer[17:24]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.

Sentence 2: Communication   stakeholder management

Sentence 3: Act as the key contact person for strategic stakeholders.

Sentence 4: Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.

Sentence 5: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.

Sentence 6: Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 7: Project Controlling



In [113]:
# nltk_tokenizer = []

# for job_sentence in nltk_sentencizer:
#     for token in word_tokenize(job_sentence):
#         if len(token) != 0 and token != '...' and token.lower() not in set(stopwords.words('english')) and token.lower() not in list(string.punctuation):
#             nltk_tokenizer.append(token)


In [114]:
nltk_tokenizer = [
    token 
    for job_sentence in nltk_sentencizer 
    for token in word_tokenize(job_sentence) 
    if len(token) != 0 
    and token != '...' 
    and not token.lower() in set(stopwords.words('english')) 
    and not token.lower() in list(string.punctuation) 
]

In [115]:
'...' in nltk_tokenizer

False

In [116]:
len(nltk_tokenizer)

915

In [117]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [118]:
custom_punct_chars = make_custom_punct_chars()

In [119]:
custom_punct_chars

[':',
 ': ',
 '|',
 '| ',
 '\n',
 '\n ',
 '\n\n',
 '\n \n ',
 ',,',
 ', , ',
 '\n\n\n',
 '\n \n \n ',
 ',,,',
 ', , , ',
 '\n\n\n\n',
 '\n \n \n \n ',
 ',,,,',
 ', , , , ']

In [147]:
# Set up Spacy
import spacy
from spacy.symbols import ORTH, LEMMA

# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_trf')


In [148]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [149]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [150]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_char = pickle.load(f)

In [151]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [152]:
# nlp = spacy.load('en_core_web_sm')
# nlp.tokenizer.add_special_case('incl.', [{ORTH: 'incl', LEMMA: 'including'}])


In [153]:
# from spacy.symbols import ORTH, LEMMA

# special_cases_dict = {
#     'incl.': [{65: 'incl', 67: 'including'}],
#     'incl. ': [{65: 'incl', 67: 'including'}],
#     '(incl.': [{65: 'incl', 67: 'including'}],
#     'etc.': [{65: 'etc', 67: 'et cetera'}],
#     'etc. ': [{65: 'etc', 67: 'et cetera'}],
#     'e.g.': [{65: 'e.g', 67: 'for example'}],
#     'e.g. ': [{65: 'e.g', 67: 'for example'}],
# }

# nlp.tokenizer.rules.update(special_cases_dict)


In [154]:
# nlp.tokenizer.rules['e.g.']

In [155]:
# # Spacy sentencizer
# spacy_sentencizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencizer.append(sent)

In [156]:
# Spacy sentencizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_sentencizer = [
        sent 
        for job_description in job_descriptions 
        for sentence in nlp(job_description).sents 
        for sent in re.split(pattern, sentence.text) 
        if len(sent) != 0 
    ]

/opt/homebrew/Caskroom/miniforge/base/envs/study1_3.10/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [157]:
# # Spacy tokenizer
# spacy_tokenizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_sentence in spacy_sentencizer:
#         for token in nlp.tokenizer(job_sentence):
#             if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#                 spacy_tokenizer.append(token.text)

In [158]:
type(spacy_sentencizer[0])

str

In [159]:
len(spacy_sentencizer)

133

In [160]:
# Spacy tokenizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_tokenizer = [
        token.text 
        for job_sentence in spacy_sentencizer 
        for token in nlp.tokenizer(job_sentence) 
        if len(token) != 0 
        and not token.is_stop 
        and not token.is_punct 
        and not token.text in custom_punct_chars
    ]

In [161]:
type(spacy_tokenizer[0])

str

In [162]:
len(spacy_tokenizer)

911

In [163]:
for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_eg in sent:
        print(idx)

17


In [164]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if str_fix_eg in sent)

17

In [165]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

20
84


In [166]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if str_fix_incl.split('\(')[0] in sent)

20

In [167]:
spacy_sentencizer[18:24]

['Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.',
 'Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling']

In [168]:
for idx, sent in enumerate(spacy_sentencizer):
    if 'Power' in sent:
        print(idx)

62
128


In [169]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if 'Power' in sent)

62

In [170]:
spacy_sentencizer[63]

'Ability to travel, domestic or international, as required'

In [171]:
for idx, sentence in enumerate(spacy_sentencizer[18:24]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Communication   stakeholder management

Sentence 2: Act as the key contact person for strategic stakeholders.

Sentence 3: Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.

Sentence 4: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.

Sentence 5: Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 6: Project Controlling



In [172]:
# # Spacy tokenizer
# spacy_tokenizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_sentence in spacy_sentencizer:
# #         doc = nlp.tokenizer(job_sentence)
#         spacy_tokenizer.extend(
#             [
#                 token.text for token in nlp.tokenizer(job_sentence) 
#                 if token.text not in custom_punct_chars
#                 and not token.is_stop 
                
#             ]
#         )

In [173]:
len(spacy_tokenizer)

911

In [174]:
spacy_tokenizer[:10]

['Purpose',
 'Studio',
 'Director',
 'Special',
 'Projects',
 'adidas',
 'Digital',
 'task',
 'manage',
 'deliver']

In [175]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_trf')

matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [177]:
trigram_patterns

[[{'POS': 'VERB'}, {'POS': 'ADJ'}, {'POS': 'NOUN'}],
 [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'ADV'}],
 [{'POS': 'NOUN'}, {'POS': 'ADP'}, {'POS': 'NOUN'}]]

In [178]:
patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [179]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [180]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencizer:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)